### Создание `единой базы`, хранящей структуру с путями

#### Подключение к диску, где хранятся все декомпозиции отчетов 
`192.168.137.76`

In [10]:
import os

import json
import random
from typing import List, Dict, Any

In [9]:
def build_hierarchical_structure(root_path):
    """
    Рекурсивно строит иерархическую структуру каталогов и файлов.
    Исключает дублирование, если имя папки совпадает с именем файла `.pdf`.
    Сортирует элементы в алфавитном порядке.
    """
    structure = {"name": os.path.basename(root_path), "children": []}
    try:
        items = sorted(os.listdir(root_path))  # Сортируем элементы по имени
        pdf_files = {os.path.splitext(f)[0]: f for f in items if f.endswith(".pdf")}
        for item in items:
            item_path = os.path.join(root_path, item)
            if os.path.isdir(item_path):
                if item.startswith("Декомпозиция_"):
                    # Если папка начинается с "Декомпозиция_", её пропускаем и обрабатываем содержимое
                    structure["children"].extend(build_hierarchical_structure(item_path)["children"])
                elif item in pdf_files:
                    # Если имя папки совпадает с именем PDF, добавляем только файл
                    structure["children"].append({
                        "name": item,
                        "path": os.path.join(root_path, pdf_files[item])
                    })
                else:
                    # Если это обычная папка, добавляем её как узел
                    sub_structure = build_hierarchical_structure(item_path)
                    # Проверяем вложенные элементы на дублирование
                    if len(sub_structure["children"]) == 1 and sub_structure["children"][0]["name"] == sub_structure["name"]:
                        # Если есть дубликат имени, заменяем узел только на файл
                        structure["children"].append(sub_structure["children"][0])
                    else:
                        structure["children"].append(sub_structure)
            elif item.endswith(".pdf") and os.path.splitext(item)[0] not in [c["name"] for c in structure["children"]]:
                # Если это PDF-файл, добавляем его, если его имя не совпадает с именем существующих узлов
                structure["children"].append({"name": os.path.splitext(item)[0], "path": item_path})
    except PermissionError:
        # Игнорируем папки без доступа
        pass
    return structure

# Путь к монтированной папке
remote_path = "/Volumes/gmvo/ai_pdf_store/Base_Books/"
hierarchical_structure = build_hierarchical_structure(remote_path)

output_file = os.path.join(os.getcwd(), "hierarchical_book_structure.json")
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(hierarchical_structure, json_file, ensure_ascii=False, indent=4)

#### Обновление струкруты под задачу
- из детей берется путь родителя и поднимается на уровень
- очищаются именя разделов 

In [11]:
# Список новых названий
new_titles = [
    "Общая характеристика речного бассейна",
    "Оценка экологического состояния и ключевые проблемы речного бассейна",
    "Целевые показатели",
    "Водохозяйственные балансы и балансы загрязняющих веществ",
    "Лимиты и квоты на забор воды из водных объектов и сброс сточных вод",
    "Перечень мероприятий по достижению целевого состояния речного бассейна"
]

def update_structure(node, titles):
    if 'children' in node:
        for child in node['children']:
            update_structure(child, titles)
        
        # Проверка и обновление структуры
        for child in node['children']:
            if node['name'] == child['name'].lstrip('._'):
                node['path'] = child['path']
                node['children'].remove(child)
                break

def reorder_structure(node, titles):
    if 'children' in node:
        for child in node['children']:
            reorder_structure(child, titles)
    
    # Переместить path после name
    if 'path' in node:
        reordered_node = {'name': node['name'], 'path': node['path']}
        if 'children' in node:
            reordered_node['children'] = node['children']
        node.clear()
        node.update(reordered_node)
    
    # Обновление имени
    node['name'] = node['name'].replace('_', ' ').lstrip('.').lstrip(' ').replace('  ', ' ').replace('й', 'й')    

    # Заменяем названия Книга 1...Книга 6 на новые
    if node['name'].startswith("Книга"):
        try:
            book_number = int(node['name'].split()[1])  # Извлекаем номер книги
            if 1 <= book_number <= len(titles):  # Проверяем, что номер в пределах списка
                node['name'] = titles[book_number - 1]
        except (IndexError, ValueError):
            pass  # Если не удаётся распознать номер, оставляем название без изменений

with open('hierarchical_book_structure.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Удаление первого уровня, если он не несет смысла
if 'children' in data and data['name'] == "":
    data = data['children']

# Обновление структуры
for node in data:
    update_structure(node, new_titles)
    reorder_structure(node, new_titles)

with open('hierarchical_structure.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

### Перезапись названий всех разделов 
Из "3 Перечень мероприятий по достижению целевого сост" -> "3 Перечень мероприятий по достижению целевого состояния речного бассейна"
такие сокращения были ввиду невозможной записи полного названия в именя папок/файлов

In [21]:
def extract_titles_from_directory(directory):
    def extract_titles(data, titles):
        if isinstance(data, list):
            for item in data:
                extract_titles(item, titles)
        elif isinstance(data, dict):
            if 'title' in data:
                titles.append(data['title'])
            if 'subsections' in data:
                extract_titles(data['subsections'], titles)

    all_titles = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    titles = []
                    extract_titles(data, titles)
                    all_titles.extend(titles)
    return all_titles

def update_names(node, titles):
    if 'children' in node:
        for child in node['children']:
            update_names(child, titles)
    
    if 'name' in node:
        for title in titles:
            if node['name'] in title:
                print(f"Updating name from '{node['name']}' to '{title}'")
                node['name'] = title
                break

# Список бассейнов и соответствующих директорий
basins = {
    "Бассейн Волга": '../../data/JSON/Волга/Исходные',
    "Бассейн Дон": '../../data/JSON/Дон/Исходные',
    "Бассейн Кубань": '../../data/JSON/Кубань/Исходные',
    "Бассейн Обь": '../../data/JSON/Обь/Исходные',
    "Бассейн Печора": '../../data/JSON/Печора/Исходные',
    "Бассейн Сура": '../../data/JSON/Сура/Исходные',
    "Бассейн Терек": '../../data/JSON/Терек/Исходные',
    "Бассейн Урал": '../../data/JSON/Урал/Исходные'
}

# Загрузка JSON файла
with open('hierarchical_structure.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Обработка каждого бассейна
for basin_name, directory_path in basins.items():
    # Извлечение titles из директории
    all_titles = extract_titles_from_directory(directory_path)
    all_titles = set(all_titles)

    print('____'* 20)
    print(all_titles)
    # Обновление имен в структуре
    for node in data:
        if node['name'] == basin_name:
            update_names(node, all_titles)

# Сохранение обновленного JSON файла
with open('hierarchical_structure.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

________________________________________________________________________________
{'2.2.2 Качество воды по санитарно-микробиологическим показателям', '3.2. Целевые показатели развития системы мониторинговых наблюдений в бассейне р. Волги', '3.1. Выявление и формулировка проблем экологического состояния водных объектов', 'Российская Федерация', '6.3. Гидротехнические сооружения', '3.2.1. Наводнения.', '3.5.1. Ранжирование проблем', '3.1.1 Угроза наращивания темпов "цветения" водохранилищ цианобактериями (сине-зелеными водорослями).', '3.3. Водообеспеченность.', '2.2. Качество воды по пунктам наблюдений', 'ВВЕДЕНИЕ', 'Раздел 3. Институциональные мероприятия по достижению целевого состояния рассматриваемой территории Волжского бассейна на период 2011 – 2020 годы', '2.2 Расчетные водохозяйственные балансы для года 50% обеспеченности', '2.2.1. Природное качество вод – как целевой показатель состояния вод Волжского каскада водохранилищ.', 'Раздел 4. Мероприятия по улучшению оперативного управ

### Добавление текста для интерактивного исползования

In [22]:
# Тексты для каждого уровня структуры
texts = {
    1: [  # Уровень "Глава / Раздел"
        "Выберите всю интересующую Вас информацию из представленного списка разделов:",
        "Начните с раздела, который соответствует Вашему интересу:",
        "Ниже представлен список доступных разделов:",
        "Перейдите к разделу, который подходит Вашему запросу:",
        "Разделы помогут Вам быстро найти нужную информацию:",
        "Ознакомьтесь с предложенными разделами для поиска данных:",
        "Выберите подходящий раздел для более детального изучения:",
        "Исследуйте разделы, чтобы найти необходимую информацию:",
        "Список разделов создан для Вашего удобства. Начните выбор:",
        "Рассмотрите предложенные разделы и выберите наиболее подходящий:",
    ],
    2: [  # Уровень "Подраздел"
        "Выберите подраздел для уточнения информации:",
        "Подразделы содержат более узкоспециализированные данные:",
        "Для детализации информации выберите нужный подраздел:",
        "Подразделы помогут глубже изучить интересующую Вас тему:",
        "Выберите подраздел, чтобы продолжить работу с информацией:",
        "Список подразделов подготовлен для Вашего удобства:",
        "Доступные подразделы позволяют сузить область поиска:",
        "Выберите подраздел для конкретизации данных:",
        "Ознакомьтесь с подразделами для дальнейшего изучения:",
        "Список подразделов предложен ниже. Выберите подходящий:",
    ],
    3: [  # Уровень "Пункт"
        "Выберите пункт для более детального изучения:",
        "Каждый пункт предоставляет точную информацию по теме:",
        "Пункты содержат ключевые аспекты выбранного подраздела:",
        "Ознакомьтесь с пунктами для детального анализа:",
        "Выберите пункт, чтобы найти нужные данные:",
        "Доступные пункты представлены ниже для Вашего удобства:",
        "Выберите пункт, чтобы перейти к подробной информации:",
        "Список пунктов подготовлен для поиска нужных данных:",
        "Изучите пункты, чтобы получить полное представление о теме:",
        "Пункты позволят Вам уточнить интересующую информацию:",
    ],
    4: [  # Уровень "Подпункт"
        "Выберите подпункт для конкретизации данных:",
        "Каждый подпункт представляет узкую тему в рамках пункта:",
        "Подпункты содержат дополнительные сведения по теме:",
        "Перейдите к подпункту для более глубокого изучения:",
        "Список подпунктов поможет уточнить Ваш запрос:",
        "Выберите подходящий подпункт из приведённого списка:",
        "Подпункты позволяют найти дополнительные аспекты темы:",
        "Исследуйте подпункты для получения дополнительной информации:",
        "Выберите подпункт, чтобы перейти к следующим деталям:",
        "Список подпунктов приведён ниже. Выберите нужный:",
    ],
    5: [  # Уровень "Подподпункт"
        "Выберите подподпункт для окончательной детализации данных:",
        "Подподпункты содержат самую узкую специализацию по теме:",
        "Выберите подподпункт, чтобы завершить анализ информации:",
        "Список подподпунктов представлен для уточнения темы:",
        "Перейдите к подподпункту для завершения поиска данных:",
        "Подподпункты позволяют сосредоточиться на деталях темы:",
        "Выберите подподпункт из приведённого списка для анализа:",
        "Ознакомьтесь с подподпунктами для завершения изучения:",
        "Список подподпунктов поможет найти все нужные данные:",
        "Подподпункты представлены ниже для окончательного выбора:",
    ],
}

In [17]:
def add_texts_to_structure(node, level):
    if 'children' in node:
        if level in texts:
            text = random.choice(texts[level])
            # Вставляем текст после поля 'name'
            node_with_text = {'name': node['name'], 'text': text}
            if 'path' in node:
                node_with_text['path'] = node['path']
            node_with_text['children'] = node['children']
            node.clear()
            node.update(node_with_text)
        
        for child in node['children']:
            add_texts_to_structure(child, level + 1)

# Загрузка JSON файла
with open('hierarchical_structure.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Добавление текстов в структуру
for node in data:
    add_texts_to_structure(node, 1)

# Сохранение обновленного JSON файла
with open('hierarchical_structure.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [23]:
def sort_sections(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Recursively sort sections by the number in "Раздел X" in their names.
    """
    def extract_number(name: str) -> int:
        # Extract number from the section name "Раздел X"
        import re
        match = re.search(r"Раздел\s*(\d+)", name)
        if match:
            return int(match.group(1))
        return float('inf')  # For names without "Раздел X", place them last

    for item in data:
        if "children" in item:
            # Recursively sort the children
            item["children"] = sort_sections(item["children"])
    # Sort the current level
    data.sort(key=lambda x: extract_number(x.get("name", "")))
    return data

# Считывание JSON из файла
input_file = "hierarchical_structure.json"
output_file = "hierarchical_structure.json"

with open(input_file, "r", encoding="utf-8") as file:
    data = json.load(file)

# Сортировка
sorted_data = sort_sections(data)

# Сохранение результата в файл
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(sorted_data, file, ensure_ascii=False, indent=4)

print(f"Сортированный JSON сохранен в {output_file}")

Сортированный JSON сохранен в hierarchical_structure.json
